In [1]:
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
path = './data/'

train = pd.read_csv(path+"train_preprocessed.csv")
train_errx = pd.read_csv(path+"train_errx_preprocessed.csv")
test = pd.read_csv(path+"test_preprocessed.csv")
sub = pd.read_csv(path + 'sample_submission.csv')

train.shape, train_errx.shape, test.shape

((2952, 62), (2896, 62), (1008, 61))

In [9]:
# train, test 데이터세트 합치기

all_df = pd.concat([train.drop("등록차량수",axis=1), test])
print(all_df.shape)
print(all_df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,-1]].isnull().sum() )

all_df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,-1]].head(8)

(3960, 61)
단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
자격유형          0
임대보증금       767
임대료         776
지하철         249
버스정류장         0
단지내주차면수       0
O             0
dtype: int64


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,O
0,ERR04_2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,0.0
1,ERR04_2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000.0,103680.0,0.0,3.0,1425.0,0.0
2,ERR04_2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,0.0
3,ERR04_2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,0.0
4,ERR04_2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,0.0
5,ERR04_2483,900,아파트,경상북도,국민임대,51.95,89,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,0.0
6,ERR04_2483,900,아파트,경상북도,국민임대,51.95,135,38.0,A,27304000.0,184330.0,0.0,3.0,1425.0,0.0
7,ERR04_2483,900,아파트,경상북도,국민임대,59.88,86,38.0,A,30357000.0,214270.0,0.0,3.0,1425.0,0.0
